# Mongo Queries

In [2]:
import pymongo

mongoclient = pymongo.MongoClient("mongodb://localhost:27017/")
dblp = mongoclient["testdb"]
articles_collection = dblp["articles"]
proceeding_collection = dblp["proceedings"]
inproceeding_collection = dblp["inproceedings"]

#### E1: Who is the publisher of the PODS conference proceedings?

In [3]:
#Query E1
doc = proceeding_collection.find({"booktitle": "PODS"}, {"publisher" : 1})

print(doc[0])

{'_id': ObjectId('638d07005bdfe572e95bd5b5'), 'publisher': 'ACM'}


#### E2: What are the titles of the articles that Martin Grohe wrote in the Theory of Computing Systems journal? (Sort in alphabetic order)


In [11]:
docs = articles_collection.find({"author" : "Martin Grohe", "journal": {"$regex" : "^Theor"}}, {"title" : 1}).sort("title")
for doc in docs:
    print(doc.get('title'))


Database Query Processing Using Finite Cursor Machines.
Learnability and Definability in Trees and Similar Structures.
Machine-based methods in parameterized complexity theory.
The complexity of partition functions.
Tight Lower and Upper Bounds for the Complexity of Canonical Colour Refinement.
Tight lower bounds for query processing on streaming and external memory data.
